# The Core: A different view on explanation

On this page another game theoretic concept *the core* is introduced and how it can be applied for explanation.
Mainly applied in the field of economics the core is a newly introduced concept in the field of explanation for AI researchers.

## Example: Writing a Paper
Let us consider three AI researchers Alice, Bob and Charlie which work together on a paper.
The paper is about a new method to explain AI models and would win the best paper award at the next conference.
Winning the best paper award would grant them a \$500 price.
Despite their long friendship they would like to have a fair share of the prize money.


As they are AI researchers they are familiar with the concept of *Shapley values*, being a fair way to distribute the prize money.
They have previously recorded their best paper award prizes:


| Authos          | Award |
|-----------------|---|
|                 | 0 |
| Alice           | 0 |
| Bob             | 0 |
| Charlie             | 0 |
| Alice, Bob      | 500 |
| Alice, Charlie      | 400 |
| Bob, Charlie        | 350 |
| Alice, Bob, Charlie | 500 |

Based on these values they would like to distribute the prize money in a fair way via the Shapley values.
Running the calculations they get the following values:

| Authos          | Shapley Value |
|-----------------|---|
| Alice           | 200 |
| Bob             | 175 |
| Charlie             | 125 |

An inherent assumption when computing the Shapley value is that Alice, Bob and Charlie really work together.
Yet it might be the case that Alice and Bob realize they would get a higher payoff if they would exclude Charlie, namely \$250 each.
The concept of the core is now introduced to prevent such behavior.

## The Core
The main idea of the core is to distribute the money in such a way, that there is no incentive for anybody to leave the group.
In our example the core would be the set of all possible distributions of the prize money, where Alice, Bob and Charlie will work together.
The underlying concept capturing "leave the group" is *stability*.
We say a payoff distribution $\psi(N,\nu)$ of a game (N,\nu) is stable iff.
$$
\forall S\subseteq N. \sum_{i \in S} \psi(S,\nu)_i \geq v(S)
$$
where $\psi(S,\nu)_i$ is the payoff of player $i$ in the coalition $S$.
Stability induces no need to leave the group as each player gets what he would get in a smaller group.
The core is then defined as the set of all stable payoff distributions.
$$
    \mathcal{C} := \{ \psi(N,\nu) | \forall S\subseteq N. \sum_{i \in S} \psi(N,\nu)_i \geq v(S) \land \sum_{i \in N} \psi(N,\nu)_i = \nu(N) \}
$$



## The Core of Writing a Paper
Let us now compute the core for our example.
To do so we must find at least one payoff distribution, which is stable.
Let us abbreviate the authors payoffs as $a$,$b$ and $c$.
\begin{align}
   (I)&: a + b \geq 500 \\
   (II)&: a + c \geq 400 \\
   (III)&: b + c \geq 350 \\
   (IV)&:  a + b + c  = 500
\end{align}
From (I) and (IV) we get $c \leq 0$ from which $c = 0$ must follow, due to $c \geq \nu(\{c\})=0$.
Now we can substitute $c=0$ into (II) and (III) to get $a \geq 400$ and $b \geq 350$.
Thus $ a + b \geq 750$ which is a contradiction to (IV), showing the **core is empty**.


## The least-Core
Due to the core being empty, we can introduce a new concept called the least-core.
Firstly lets define the **e-core** to be the payoff distributions, which are stable up to a given $e$.
$$
    \mathcal{C_e} := \{ \psi(N,\nu) | \forall S\subseteq N. \sum_{i \in S} \psi(N,\nu)_i + e \geq v(S) \land \sum_{i \in N} \psi(N,\nu)_i = \nu(N) \}
$$
Then the **least-core** is the minimal $e$ such that $\mathcal{C_e}$ is not empty.

In comparison to the Shapley value the (least-)core may contain multiple payoff distributions.
Often we are interested in a single payoff distribution, for which we choose the **egalitarian-least-core**.
Given the least-core $C_{e^*}$ the egalitarian-least-core is $x \in C_{e^*}$ with minimal $||x||_2$.
It is important to note that the least-core is **always exists** whereas the core is potentially empty.


## The least-Core of Writing a Paper
Computing the least-core for our example can become quite tedious as solving linear inequalities is required.
To simplify the process we use the *ExactComputer* from the *shapiq* package.


In [21]:
import numpy as np
from shapiq.exact import ExactComputer
from shapiq.games.base import Game


# Define the PaperGame as described above
class PaperGame(Game):

    def __init__(self) -> None:
        super().__init__(n_players=3, normalize=True, normalization_value=0)

    def value_function(self, coalitions: np.ndarray) -> np.ndarray:
        coalition_values = {
            (): 0,
            (0,): 0,
            (1,): 0,
            (2,): 0,
            (0, 1): 500,
            (0, 2): 400,
            (1, 2): 350,
            (0, 1, 2): 500,
        }

        values = np.array([coalition_values[tuple(np.where(x)[0])] for x in coalitions])

        return values


paper_game = PaperGame()

# Compute the egalitarian least core
exact_computer = ExactComputer(n_players=3, game_fun=paper_game)
egalitarian_least_core = exact_computer("ELC")

The egalitarian least core values can then be viewed with `.dict_values`.

In [22]:
egalitarian_least_core.dict_values

{(0,): 233.3333333333333, (1,): 183.33333333333337, (2,): 83.33333333333334}

The minimal $e$ is stored in `exact_computer._elc_stability_subsidy`.

In [23]:
exact_computer._elc_stability_subsidy

83.33333333333334